# Picks and Bans
## General Information

Author: Patrick McNamee
Date: 11/25/2019

## Introduction

DOTA 2 has a drafting round before the game which can go to influence the outcomes of the game. Ideally, teams would be able to choose all the heros that would benefit them and ban all heros which would benefit the other team. However both teams are competiting against each other and trying to minimize the others gain while maximizing their own gain. The idea behind this notebook is to analyze previous matches to best predict future picks and bans. It is assumed that human players already now very subtle aspects of the game and their actions would best predict the optimal pick/ban strategy. Hence a bayesian statistics approach will be used to attempt to predict future moves to provide a team with more information by providing likely outcomes of draft choices.

## Data Engineering

First thing to do is to inport the data and them have the picks and bans drafting into a singular dataframe.


In [1]:
import pandas as pd

df = pd.read_csv(r'./international_raw_match_full.csv')
df = df['picks_bans']
df.head()
df.tail()

2936                                                  NaN
2937                                                  NaN
2938                                                  NaN
2939                                                  NaN
2940    [{'is_pick': False, 'hero_id': 89, 'team': 1, ...
Name: picks_bans, dtype: object

In [2]:
draft = eval(df.loc[0])
print(draft)
for i in range(len(draft)):
    print('Pick = %r\tTeam=%i' % (draft[i]['is_pick'], draft[i]['team']))

[{'is_pick': False, 'hero_id': 52, 'team': 0, 'order': 0, 'ord': 0, 'match_id': 4986461644}, {'is_pick': False, 'hero_id': 97, 'team': 1, 'order': 1, 'ord': 1, 'match_id': 4986461644}, {'is_pick': False, 'hero_id': 55, 'team': 0, 'order': 2, 'ord': 2, 'match_id': 4986461644}, {'is_pick': False, 'hero_id': 73, 'team': 1, 'order': 3, 'ord': 3, 'match_id': 4986461644}, {'is_pick': False, 'hero_id': 58, 'team': 0, 'order': 4, 'ord': 4, 'match_id': 4986461644}, {'is_pick': False, 'hero_id': 103, 'team': 1, 'order': 5, 'ord': 5, 'match_id': 4986461644}, {'is_pick': True, 'hero_id': 91, 'team': 0, 'order': 6, 'ord': 6, 'match_id': 4986461644}, {'is_pick': True, 'hero_id': 66, 'team': 1, 'order': 7, 'ord': 7, 'match_id': 4986461644}, {'is_pick': True, 'hero_id': 57, 'team': 1, 'order': 8, 'ord': 8, 'match_id': 4986461644}, {'is_pick': True, 'hero_id': 19, 'team': 0, 'order': 9, 'ord': 9, 'match_id': 4986461644}, {'is_pick': False, 'hero_id': 82, 'team': 0, 'order': 10, 'ord': 10, 'match_id': 4

The first draft shows the drafting order, at least for this notebook. There are 3 bans groupings and 3 picking groupings with some alternations between teams on ordering. This ordering above is the draft order used in past international tournaments although there has been a shift so that there is more bans. Future work will have to modified to fit this new style but currently there is not enough data to use the new format.

While we have the draft in a dictionary sort of structure, we will convert it into a dataframe for easier model building.

In [3]:
import numpy as np

#transform data from array into a dataframe
matches = pd.DataFrame()

draft_order = ['DB1', 'RB1', 'DB2', 'RB2',\
               'DP1', 'RP1', 'RP2', 'DP2',\
               'DB3', 'RB3', 'DB4', 'RB4',\
               'RP3', 'DP3', 'RP4', 'DP4',\
               'RB6', 'DB6',\
               'RP5', 'DP5']

for i in range(df.index[-1] + 1):
    if type(df.loc[i]) is not float:
        info = eval(df.loc[i]) #dictionary
        if len(info) == 20:
            matches.loc[i,'id'] = int(info[0]['match_id'])
            for j in range(len(info)):
                matches.loc[i,draft_order[j]] = int(info[j]['hero_id'])
        else:
            continue

#Displaying results
matches.dropna()
#print(matches.shape)
matches.reset_index(drop=True)

,id,DB1,RB1,DB2,RB2,DP1,RP1,RP2,DP2,DB3,...,DB4,RB4,RP3,DP3,RP4,DP4,RB6,DB6,RP5,DP5
0,3.372726e+09,60.0,53.0,107.0,91.0,7.0,40.0,27.0,31.0,10.0,...,73.0,29.0,16.0,55.0,96.0,36.0,1.0,74.0,8.0,43.0
1,3.372676e+09,53.0,107.0,91.0,60.0,31.0,7.0,40.0,104.0,88.0,...,10.0,90.0,68.0,16.0,36.0,39.0,63.0,77.0,1.0,73.0
2,3.372623e+09,60.0,91.0,107.0,7.0,53.0,88.0,4.0,28.0,59.0,...,40.0,2.0,5.0,31.0,65.0,54.0,10.0,39.0,95.0,25.0
3,3.372456e+09,91.0,68.0,90.0,60.0,7.0,53.0,86.0,65.0,18.0,...,95.0,59.0,71.0,30.0,40.0,10.0,99.0,1.0,43.0,109.0
4,3.372387e+09,68.0,91.0,60.0,36.0,7.0,31.0,13.0,53.0,61.0,...,59.0,107.0,4.0,5.0,100.0,40.0,10.0,94.0,73.0,109.0
5,3.372270e+09,68.0,91.0,60.0,36.0,7.0,31.0,13.0,90.0,61.0,...,59.0,43.0,107.0,86.0,54.0,4.0,53.0,65.0,80.0,35.0
6,3.370364e+09,68.0,91.0,60.0,90.0,7.0,65.0,23.0,40.0,31.0,...,74.0,18.0,111.0,59.0,43.0,50.0,57.0,99.0,61.0,98.0
7,3.370308e+09,68.0,91.0,60.0,90.0,7.0,97.0,23.0,36.0,80.0,...,61.0,95.0,27.0,31.0,9.0,10.0,53.0,1.0,55.0,44.0
8,3.370213e+09,88.0,43.0,40.0,65.0,60.0,107.0,4.0,50.0,109.0,...,81.0,13.0,87.0,99.0,2.0,10.0,38.0,76.0,57.0,11.0
9,3.370154e+09,88.0,43.0,40.0,65.0,60.0,107.0,4.0,50.0,109.0,...,81.0,13.0,37.0,38.0,99.0,56.0,94.0,39.0,17.0,45.0


There is just under 2000 data references with 20 states each. The reason to use the international and not the professional data set is that the international is included within the professional. Hence working with the professional set which has more data will act as a testing environment for the model. The next step is to build the bayesian probability models which is done through nested dictionaries.

In [4]:
import pickle

single_picks = dict() #empty dictionary for turn-by-turn
block_picks = dict() #empty dictionary for block turns

def updateDictionary(dictionary, character):
    #Initial dictionary
    if dictionary == {}:
        dictionary.update({character : {'count': 1, 'next' : {}}})
    # Existing character
    elif character in dictionary.keys():
        dictionary[character]['count'] = dictionary[character]['count'] + 1 #update counter
    #Add in character
    else:
        dictionary.update({character: {'count' : 1, 'next' : {}}}) #create new entry

    return dictionary[character]['next']

def updateFinalDictionary(dictionary, character):
    #Initial dictionary
    if dictionary == {}:
        dictionary.update({character : {'count': 1}})
    # Existing character
    elif character in dictionary.keys():
        dictionary[character]['count'] = dictionary[character]['count'] + 1 #update counter
    #Add in character
    else:
        dictionary.update({character: {'count' : 1}}) #create new entry

#Individual single picks
for i, row in matches.iterrows():
    for i in range(len(draft_order)):
        if i == 0:
            currentround = updateDictionary(single_picks, row[draft_order[0]])
        elif i == len(draft_order) - 1:
            updateFinalDictionary(currentround, row[draft_order[-1]])
        else:
            currentround = updateDictionary(currentround, row[draft_order[i]])

#block picks
block_order = [['DB1', 'RB1', 'DB2', 'RB2'],\
               ['DP1', 'RP1', 'RP2', 'DP2'],\
               ['DB3', 'RB3', 'DB4', 'RB4'],\
               ['RP3', 'DP3', 'RP4', 'DP4'],\
               ['RB6', 'DB6'],\
               ['RP5', 'DP5']]

for i, row in matches.iterrows():
    for i in range(len(block_order)):
        temp = row[block_order[i]].tolist()
        temp.sort()
        if i == 0:
            currentround = updateDictionary(block_picks, str(temp))
        elif i == len(block_order) - 1:
            updateFinalDictionary(currentround, str(temp))
        else:
            currentround = updateDictionary(currentround, str(temp))

#pickle.dump(picks, open(r'./models/frequency-dictionary.p','wb+'))

There were two models done, picking/ban block model and an individual state model. The next process is to make the ability to track through the states of the game as they evolve and give predictions and probabilities to estimate the likelihood for an individual state path. In this we use a Laplace smoother for probability if only because some paths are missing from the training data. The Laplace smoother offers a way to give non-zero probabilities for events that have not been seen before.

In [56]:
import numpy as np 

#Additive smoothing probability
def laplaceSmoothedProbability(dictionary, character, N, alpha=1, d=123):
    if character not in dictionary.keys():
        x = 0.0
    else:
        x = dictionary[character]['count']

    return (x + alpha)/(N + alpha*d)
    
#Two step path probability
def probXiGivenXi1Xi2(Xi2, Xi1, Xi, N, history, alpha=1, n_original_groups=123):
    #Probability of going to xj from xi-2
    pXi1GivenXi2 = laplaceSmoothedProbability(Xi2, Xi1, N, (n_original_groups - len(history)))
    #Probability of xj to xi
    if Xi1 not in Xi2.keys():
        pXiGivenXi1 = alpha/(n_original_groups - len(history))
    else:
        pXiGivenXi1 = laplaceSmoothedProbability(Xi2[Xi1]['next'], Xi, Xi2[Xi1]['count'],\
                                           alpha, n_original_groups - (len(history) + 1))
    return pXi1GivenXi2*pXiGivenXi1
    
#Two step probability
def probXiGivenXi2(Xi2, Xi, N, history, alpha=1, n_original_groups=123):
    probability = 0;
    for i in range(1,n_original_groups+1):#Dota2 Characters
        if i not in history: #Can't pick any previous ones
            #Probability of going to xj from xi-2
            pxjxi2 = laplaceSmoothedProbability(Xi2, i, N, (n_original_groups - len(history)))
            #Probability of xj to xi
            if i not in Xi2.keys():
                pxixj = alpha/(n_original_groups - len(history))
            else:
                pxixj = laplaceSmoothedProbability(Xi2[i]['next'], Xi, Xi2[i]['count'],\
                                                   alpha, n_original_groups - (len(history) + 1))
            
            #Add in probability of path to total probability (exclusive events)
            probability = probability + pxixj*pxjxi2
            
    return probability

#Most likly subdictionary to follow
def mostLikelyXiDictionary(Xi2, N, history, alpha = 1, n_original_groups=123):
    rounds_player = len(history)
    
    #Determine most likely granchild
    Xi_probabilites = np.zeros(n_original_groups)
    for Xi in range(1, n_original_groups+1):
        if Xi not in history:
            Xi_probabilites[Xi-1] = probXiGivenXi2(Xi2, Xi, N, history, alpha, n_original_groups)
        else:
            Xi_probabilites[Xi-1] = np.NINF #impossibility
    Xi_mle = np.argmax(Xi_probabilites) + 1#shifting to 1 index
    
    #Determine most likely X_{i-1} dictionary
    Xi1_probabilities = np.zeros(n_original_groups)
    for Xi1 in range(1, n_original_groups+1):
        if Xi1 not in history:
            Xi1_probabilities[Xi1-1] = probXiGivenXi1Xi2(Xi2, Xi1, Xi_mle, N, history, alpha, n_original_groups)
        else:
            Xi1_probabilities[Xi1-1] = np.NINF #impossibility
    Xi1_mle = np.argmax(Xi1_probabilities) + 1#shifting to 1 index
    
    if Xi1_mle not in Xi2.keys():
        key_list = []
        for key in Xi2.keys():
            key_list.append(key)
        return Xi2[key_list[0]]['next'], Xi2[key_list[0]]['count']
    else:
        return Xi2[Xi1_mle]['next'], Xi2[Xi1_mle]['count']


In [19]:
def predictMLChoice(Xi):
    maxKey = 0
    maxCount = 0
    for key in Xi.keys():
        if Xi[key]['count'] > maxCount:
            maxCount = Xi[key]['count']
            maxKey = key
    return maxKey

def predictMLPath(Xi, forwardturns):
    path = []
    if not Xi:
        return path
    else:
        for i in range(forwardturns):
            Xi1 = predictMLChoice(Xi)
            path.append(Xi1)
            if 'next' not in Xi[Xi1].keys():
                break
            else:
                Xi = Xi[Xi1]['next']
        return path

def predictMLPathProbability(Xi, forwardturns, N, alpha, d):
    probability = 1.0
    path = predictMLPath(Xi, forwardturns)
    if not path:
        return probability
    else:
        for i, Xi1 in enumerate(path):
            probability = probability*laplaceSmoothedProbability(Xi, Xi1, N, alpha, d-i)
            if Xi1 != path[-1]:
                N = Xi[Xi1]['count']
                Xi = Xi[Xi1]['next']
        return probability

print(predictMLPath(single_picks, 21))
print(predictMLPathProbability(single_picks, 21, 1911, 0, 123))
print(predictMLPathProbability(single_picks, 21, 1911, 1, 123))
print(predictMLPathProbability(single_picks, 21, 1911, 2, 123))

[91.0, 65.0, 54.0, 73.0, 55.0, 66.0, 86.0, 72.0, 49.0, 97.0, 76.0, 92.0, 9.0, 13.0, 1.0, 79.0, 88.0, 38.0, 14.0, 16.0]
0.0005232862375719519
3.2138196976753435e-33
8.706422769796931e-36


Based on the training data, the most likely path is shown above along with probablities for alpha values of 0, 1, and 2. We will compare the model based on training data against the actual test data.

In [14]:
df = pd.read_csv(r'./premier_raw_match_full.csv')
df = df['picks_bans']

premier_matches = pd.DataFrame()
for i in range(df.index[-1] + 1):
    if type(df.loc[i]) is not float:
        info = eval(df.loc[i]) #dictionary
        if len(info) == 20:
            #premier_matches.loc[i,'id'] = int(info[0]['match_id'])
            for j in range(len(info)):
                premier_matches.loc[i,draft_order[j]] = int(info[j]['hero_id'])
        else:
            continue

#Displaying results
premier_matches.dropna()
premier_matches.reset_index(drop=True)

,DB1,RB1,DB2,RB2,DP1,RP1,RP2,DP2,DB3,RB3,DB4,RB4,RP3,DP3,RP4,DP4,RB6,DB6,RP5,DP5
0,31.0,60.0,40.0,58.0,3.0,86.0,71.0,66.0,78.0,61.0,36.0,53.0,97.0,16.0,106.0,47.0,70.0,44.0,20.0,12.0
1,3.0,31.0,114.0,60.0,71.0,58.0,84.0,45.0,61.0,54.0,69.0,8.0,78.0,9.0,20.0,98.0,106.0,4.0,25.0,35.0
2,7.0,86.0,40.0,97.0,60.0,91.0,18.0,45.0,112.0,69.0,31.0,61.0,71.0,26.0,65.0,8.0,47.0,70.0,36.0,4.0
3,7.0,86.0,40.0,71.0,60.0,66.0,53.0,97.0,106.0,114.0,61.0,4.0,107.0,112.0,72.0,44.0,98.0,47.0,99.0,49.0
4,86.0,7.0,71.0,61.0,60.0,40.0,106.0,53.0,20.0,97.0,47.0,66.0,31.0,75.0,69.0,36.0,4.0,82.0,74.0,12.0
5,58.0,31.0,60.0,40.0,53.0,71.0,87.0,3.0,16.0,18.0,107.0,55.0,36.0,7.0,4.0,114.0,106.0,65.0,9.0,61.0
6,31.0,60.0,107.0,58.0,84.0,3.0,53.0,18.0,47.0,40.0,65.0,63.0,39.0,91.0,16.0,49.0,55.0,70.0,78.0,98.0
7,58.0,31.0,84.0,60.0,40.0,3.0,53.0,85.0,13.0,69.0,39.0,43.0,62.0,71.0,52.0,65.0,54.0,63.0,109.0,12.0
8,91.0,60.0,66.0,51.0,53.0,40.0,7.0,86.0,43.0,45.0,39.0,31.0,90.0,88.0,18.0,54.0,47.0,69.0,13.0,61.0
9,91.0,60.0,66.0,51.0,53.0,40.0,7.0,86.0,43.0,45.0,68.0,36.0,31.0,71.0,114.0,39.0,9.0,61.0,4.0,69.0


In [58]:
test_draft = premier_matches.loc[premier_matches.index.tolist()[0]]

alpha = 1
N = 1911
d = 123

current_round = single_picks
history = []

print('Round:\tPredicted\tActual\tMLEprob\tActualprob')

for i in range(len(draft_order)):
    #Current round
    Xi = test_draft[draft_order[i]] #actual
    history.append(Xi)
    XiMLE = predictMLPath(current_round, 1)[0] #predicted
    probXi = laplaceSmoothedProbability(current_round, Xi, N, alpha, d - i)
    probXiMLE = laplaceSmoothedProbability(current_round, XiMLE, N, alpha, d - i)
    #Output
    print('{0}:\t{1}\t{2}\t{3}\t{4}'.format(i+1, XiMLE, Xi, probXiMLE, probXi))
    
    #Dictionary Update
    if i != (len(draft_order) - 1):
        if Xi not in current_round.keys():
            current_round, N = mostLikelyXiDictionary(current_round, N, history, alpha, 123)
        else:
            N = current_round[Xi]['count']
            current_round = current_round[Xi]['next']

Round:	Predicted	Actual	MLEprob	Actualprob
1:	91.0	31.0	0.10668633235004917	0.004424778761061947
2:	91.0	60.0	0.023076923076923078	0.007692307692307693
3:	88.0	40.0	0.01639344262295082	0.00819672131147541
4:	82.0	58.0	0.01652892561983471	0.008264462809917356
5:	13.0	3.0	0.016666666666666666	0.008333333333333333
6:	16.0	86.0	0.01680672268907563	0.008403361344537815
7:	27.0	71.0	0.01694915254237288	0.00847457627118644
8:	6.0	66.0	0.017094017094017096	0.008547008547008548
9:	53.0	78.0	0.017241379310344827	0.008620689655172414
10:	61.0	61.0	0.017391304347826087	0.017391304347826087
11:	9.0	36.0	0.017543859649122806	0.008771929824561403
12:	44.0	53.0	0.017699115044247787	0.008849557522123894
13:	55.0	97.0	0.017857142857142856	0.008928571428571428
14:	111.0	16.0	0.018018018018018018	0.009009009009009009
15:	77.0	106.0	0.01818181818181818	0.00909090909090909
16:	65.0	47.0	0.01834862385321101	0.009174311926605505
17:	94.0	70.0	0.018518518518518517	0.009259259259259259
18:	106.0	44.0	0.01869158

The prediction tended to be an overestimation of a round in probability although it did get round 10 correct. This may be a product of that multiple heros act in similiar roles so looking a an embedding of the heros would reduce the error. Another issues is with the probability. With such a large state space, it is hard to get an estimation of accuraccy and so a validation would require a lot more data points.